# Trying to find possible snorkel labelling functions

In [2]:
import sys
sys.path.append('../')
import pandas as pd
import nltk
from mongo_class import Mongo
from collections import Counter
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Importing the data

In [2]:
annotations = Mongo(db_name='annotations', collection_name='annotations')
transcripts = Mongo(db_name='videos', collection_name='diarization')

## Note!
one of the first things i did was explore what unigrams and bigrams were most common in both the comments and descriptions and use my understanding to decide which of them would be indicative of a correlation. I will replicate that here

## Exploring the most common unigrams and bigrams for different labels

## Looking at correlations between statistics and labels

In [3]:
videos = Mongo(db_name='videos', collection_name='all')
annotations = Mongo(db_name='annotations', collection_name='self-annotated-2')

In [4]:
videos_df = pd.DataFrame(list(videos.collection.find()))
annotations_df = pd.DataFrame(list(annotations.collection.find()))
transcripts_df = pd.DataFrame(list(transcripts.collection.find()))

bind [127.0.0.1]:27017: Address already in use
channel_setup_fwd_listener_tcpip: cannot listen to port: 27017
Could not request local forwarding.
bind [127.0.0.1]:27017: Address already in use
channel_setup_fwd_listener_tcpip: cannot listen to port: 27017
Could not request local forwarding.
bind [127.0.0.1]:27017: Address already in use
channel_setup_fwd_listener_tcpip: cannot listen to port: 27017
Could not request local forwarding.


In [5]:

def get_label(x):
    label = pd.NA
    if x['Answer_is-a-call_most']:
        label = 'most'
    elif x['Answer_is-a-call_some']:
        label = 'some'
    elif x['Answer_is-a-call_none']:
        label = 'none'
    return label
annotations_df['label'] = annotations_df[['Answer_is-a-call_most', 'Answer_is-a-call_some', 'Answer_is-a-call_none']].apply(get_label, axis=1)
videos_df = videos_df[videos_df['video_id'].isin(annotations_df['Input_video_id'])]
videos_df['label'] = videos_df['video_id'].apply(lambda x: annotations_df[annotations_df['Input_video_id'] == x]['label'].values[0])
videos_df['label'].value_counts()

label
none    508
most    392
some    100
Name: count, dtype: int64

### Converting the labels to numbers
0 none

1 some

2 most

In [6]:
# convert the values in the labels column to ints
videos_df['label'] = videos_df['label'].apply(lambda x: 2 if x == 'most' else 1 if x == 'some' else 0)

In [7]:
videos_df[['label', 'likes', 'comments_count', 'views', 'duration']].corr()   

,label,likes,comments_count,views,duration
label,1.000000,-0.075236,-0.077232,-0.086617,0.183309
likes,-0.075236,1.000000,0.837512,0.895973,-0.102393
comments_count,-0.077232,0.837512,1.000000,0.857257,-0.076922
views,-0.086617,0.895973,0.857257,1.000000,-0.083349
duration,0.183309,-0.102393,-0.076922,-0.083349,1.000000


In [8]:
annotations.close()
videos.close()

## Looking at greetings in the transcripts

In [9]:
from format_diarization import format_diarization
all_transcription = pd.Series(transcripts_df['diarization'].apply(lambda x: ' '.join([y['text'] for y in format_diarization(x)])))

In [10]:
greetings_sr = pd.Series(open('greetings.txt').readlines())
greetings_sr = greetings_sr[~greetings_sr.str.contains('^[!#]')].replace('\n', '', regex=True)
# find where greetings are in the transcription
greetings_sr.apply(lambda x: all_transcription.str.contains(f'{x}[ ,.!]').values.sum()).to_csv('testing.csv')


## Sentiment

In [9]:
from format_diarization import format_diarization
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
with Mongo(db_name='videos', collection_name='diarization') as diarization:
    diarization_df = pd.DataFrame(list(diarization.collection.find()))
    diarization_df['diarization'] = diarization_df['diarization'].apply(format_diarization)
    diarization_df['diarization'] = diarization_df['diarization'].apply(lambda x: ' '.join([y['text'] for y in x]))
    diarization_df.apply(lambda x: print(x['video_id'], sia.polarity_scores(x['diarization'])), axis=1)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\flynn\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


MoJBwXBPWwU {'neg': 0.189, 'neu': 0.386, 'pos': 0.425, 'compound': 0.5423}
FvYga59ataU {'neg': 0.0, 'neu': 0.272, 'pos': 0.728, 'compound': 0.9892}
t6etw_uUoz0 {'neg': 0.359, 'neu': 0.607, 'pos': 0.034, 'compound': -0.9531}
njV-DocpqrY {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
cEOeNy5SiSY {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
kNt_77ib1no {'neg': 0.111, 'neu': 0.728, 'pos': 0.161, 'compound': -0.3094}
zfy-uc3zGWQ {'neg': 0.0, 'neu': 0.739, 'pos': 0.261, 'compound': 0.7196}
lL360OmoKH4 {'neg': 0.167, 'neu': 0.684, 'pos': 0.149, 'compound': -0.4522}
96X81tlCq9Q {'neg': 0.025, 'neu': 0.744, 'pos': 0.231, 'compound': 0.9397}
0yy4pvHYREQ {'neg': 0.126, 'neu': 0.636, 'pos': 0.238, 'compound': 0.763}
0WGns_kKXhQ {'neg': 0.092, 'neu': 0.573, 'pos': 0.335, 'compound': 0.9859}
b3vcriV2hzk {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
RIgftPyGm-A {'neg': 0.0, 'neu': 0.303, 'pos': 0.697, 'compound': 0.3182}
-8bsioNfZhk {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compou

## Finding mistakes in the current model